<a href="https://colab.research.google.com/github/jayy1809/Webscraper-CrewAI2/blob/main/SSE_GPT4o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-groq

In [2]:
!pip install crewai

In [3]:
!pip install crewai_tools

  Using cached crewai_tools-0.2.6-py3-none-any.whl (51 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached lancedb-0.5.7-py3-none-any.whl (115 kB)
  Using cached pyright-1.1.367-py3-none-any.whl (18 kB)
  Using cached pytest-8.2.2-py3-none-any.whl (339 kB)
  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
  Using cached selenium-4.21.0-py3-none-any.whl (9.5 MB)
  Using cached PyGithub-1.59.1-py3-none-any.whl (342 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl (207 kB)
  Using cached youtube_transcript_api-0.6.2-py3-none-any.whl (24 kB)
  Using cached yt_dlp-2023.12.30-py2.py3-none-any.whl (3.0 MB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached pylance-0.9.18-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.6 MB)
  Using cached ratelimiter-1.2.0.post0-py3-none-any.whl (6.6 kB)
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
  Using cached semver-3.0.2-py3-none-any.whl (17 kB)

In [4]:
!pip install langchain_community

In [5]:
!pip install colab-xterm #https://pypi.org/project/colab-xterm/
%load_ext colabxterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 1.4 MB/s eta 0:00:00


In [6]:
import os
import getpass

In [7]:
from crewai import Agent, Task, Crew

In [8]:
!pip install langchain_groq

In [9]:
from langchain_groq import ChatGroq

In [10]:
groq_api = getpass.getpass("enter groq api key : ")

enter groq api key : ··········


In [11]:
llm_groq = ChatGroq(
    api_key = groq_api,
    model="mixtral-8x7b-32768"
)

In [12]:
os.environ["SERPER_API_KEY"] =  getpass.getpass('Enter your Serper API key: ')

Enter your Serper API key: ··········


In [14]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [30]:
search_tool = SerperDevTool()
scraper_tool = ScrapeWebsiteTool()
search_tool2 = WebsiteSearchTool()

In [36]:
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

In [37]:
os.environ["OPENAI_API_KEY"] = getpass.getpass('Enter your API key: ')

Enter your API key: ··········


In [41]:
researcher = Agent(
    # llm = llm_groq,
    role="Supplier Research Specialist",
    goal="Find the suppliers for the specified product. suppliers should be of large scale suppliers we can know a supplier is large scale if there is something written like this on website : (Get Quote) and website shouldnt have prices mentioned there so you can know its a large scale supplier ",
    backstory="You are an expert in market research with 10 years of experience with a focus on supplier analysis and finding Bulk suppliers for a given product. Your goal is to find the suppliers for a given product.",
    allow_delegation=False,
    tools=[search_tool, scraper_tool,search_tool2],
    verbose=True,
    # max_iter = 8
)

# Task to search and compile supplier information
def supplier_info_task(product_name):
    return Task(
        description=f"Search the internet and find the suppliers for {product_name},suppliers should be of large scale suppliers we can know a supplier is large scale if there is something written like this on website : (Get Quote) and website shouldnt have prices mentioned there so you can know its a large scale supplier , do max 3 searches on internet and then start extracting details, and extract details of the wholesale supplier from that website like supplier's web url, name, email, and phone number and location from their website and it should be correct, Extract 5 or more such suppliers",
        expected_output="""A structured list of suppliers with the following details:
        website URL : "",
        Supplier Name: "",
        Product Name: "",
        Email: " ",
        Phone Number:""",
        agent=researcher,
        # tools=[search_tool, scraper_tool,data_extraction_tool],
        output_file="supplier_info.txt",
    )

# Create the Writer Agent
writer = Agent(
    # llm = llm_groq,
    role="Supplier Information Analyst",
    goal="Summarize supplier information into a concise report.",
    backstory="You are an expert in summarizing market research data into easily understandable reports for business use.",
    allow_delegation=False,
    verbose=True,
    max_iter = 2
)

# Task to summarize the supplier information
task2 = Task(
    description="Summarize the supplier information into a bullet point list. and include everything from information",
    expected_output="A summarized bullet point list of each supplier, including their web url, product names, prices, emails, and phone numbers and location.",
    agent=writer,
    output_file="summary_report.txt",
)

Reviewer = Agent(
    # llm = llm_groq,
    role="Quality Assurance (QA) Lead",
    goal="get the data from writer and reasearcher and then check whether the details from the website link and the output that we agents have extracted are correct or not",
    backstory="You are an expert in Quality Assurance of a product in software company you have a 13 years of experience.",
    allow_delegation=False,
    verbose=True,
    # max_iter = 2
)

task3 = Task(
    description="take the input from researcher and writer and check whether all the details in the link provided and the data extracted is correct or not if correct then only let it display other wise discard that thing",
    expected_output="Yes the data is corect so display that if not then dont display the information from writer",
    agent=writer,
    output_file="summary_report.txt",
)


# Function to kick off the research process
def get_supplier_info(product_name):
    task1 = supplier_info_task(product_name)
    crew = Crew(agents=[researcher, writer], tasks=[task1, task2], verbose=2, max_rpm = 29)
    result = crew.kickoff()
    return result

# Example usage
if __name__ == "__main__":
    product_name = input("Enter the product name: ")
    supplier_info = get_supplier_info(product_name)
    print(supplier_info)


Enter the product name: Desi ChickPeas


 [DEBUG]: == Working Agent: Supplier Research Specialist
 [INFO]: == Starting Task: Search the internet and find the suppliers for Desi ChickPeas,suppliers should be of large scale suppliers we can know a supplier is large scale if there is something written like this on website : (Get Quote) and website shouldnt have prices mentioned there so you can know its a large scale supplier , do max 3 searches on internet and then start extracting details, and extract details of the wholesale supplier from that website like supplier's web url, name, email, and phone number and location from their website and it should be correct, Extract 5 or more such suppliers


> Entering new CrewAgentExecutor chain...
I need to find large-scale suppliers of Desi ChickPeas. To identify large-scale suppliers, I will look for websites that have options like "Get Quote" and do not list prices directly. I will start by searching the internet for relevant suppliers.

Action: Search the internet
Action Input: {"s